Issues to solve
* Far too many duplicates being collected in the dataset
* Handle mispelled and redundant words
* removing mentions and hashtag
* removing links, special characters, punctuation marks
* stopwords removal
* contractions handling
* stemming and lemmatization

In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import re
import fnmatch
import string


In [12]:
tweets_df = pd.read_csv('/Users/nitanshjain/Documents/Thapar 4th Sem/Machine Learing/Machine_Learning_Project/data/20221026_235416_tweets.csv')
tweets_df.head()

,user_id,tweet_id,username,location,following,followers,twt_created_at,total_tweets,retweet_count,text,hashtags,mentions
0,100673521,1585301310026711041,sai1951,NaN,396,81,2022-10-26 16:04:20+00:00,8226,0,@DHFWKA @PMOIndia @MoHFW_INDIA @CMofKarnataka ...,"[{'text': 'Covid_19', 'indices': [392, 401]}, ...","[{'screen_name': 'DHFWKA', 'name': ""K'taka Hea..."
1,101430927,1585255112788017153,shivaramsingh,Cuttack,777,1242,2022-10-26 13:00:46+00:00,5578,0,@nipun29j @drsuniltaneja @doc_arka @docMPK @sa...,[],"[{'screen_name': 'nipun29j', 'name': 'nipun ve..."
2,1550427109,1585252333013061634,rhltiwari711,"Lucknow, India",2573,520,2022-10-26 12:49:43+00:00,4641,0,"Dear @TwitterIndia, kindly provide @verified t...","[{'text': 'VC', 'indices': [74, 77]}, {'text':...","[{'screen_name': 'TwitterIndia', 'name': 'Twit..."
3,190904456,1585231415532490753,amishradp,"दिल्ली, भारत",51,9,2022-10-26 11:26:36+00:00,243,0,@care_mediassist @royalsundaram @naveen_shahi1...,[],"[{'screen_name': 'care_mediassist', 'name': 'S..."
4,350945055,1585230709547237379,chouhanneeraj07,"Mumbai, India",879,488,2022-10-26 11:23:47+00:00,3450,0,@drchetandeshmu1 @aparanjape @JM_Scindia @mans...,[],"[{'screen_name': 'drchetandeshmu1', 'name': 'd..."


In [13]:
# tweets_df.isna().sum()

In [14]:
print('Shape of dataset before removal of duplicates is {}'.format(tweets_df.shape))
tweets_df.drop_duplicates(subset=['tweet_id'], inplace=True)
print('Shape of dataset after removal of duplicates is {}'.format(tweets_df.shape))

Shape of dataset before removal of duplicates is (1293, 12)
Shape of dataset after removal of duplicates is (218, 12)


In [15]:
tweets_df.dtypes

user_id            int64
tweet_id           int64
username          object
location          object
following          int64
followers          int64
twt_created_at    object
total_tweets       int64
retweet_count      int64
text              object
hashtags          object
mentions          object
dtype: object

In [16]:
print(tweets_df.loc[0,'text'])

@DHFWKA @PMOIndia @MoHFW_INDIA @CMofKarnataka @BSBommai @mla_sudhakar @Comm_dhfwka @MDNHM_Kar @BBMPCOMM @mangalurucorp @DDChandanaNews @PIBBengaluru @KarnatakaVarthe Covid numbers in Karnataka:
▪️New cases in State: 96
▪️New cases in B'lore: 57
▪️Positivity rate: 2.19%
▪️Discharges: 271 (B'lore-209)
▪️Deaths: 0
▪️Active cases in State: 1,953
▪️Active cases in B'lore: 1,687
▪️Tests: 4,367

#Covid_19 #COVID19


In [17]:
def  removing_hashtags_mentions(df):
    # count = 0
    for i in range(len(df)):
        
        words = df.loc[i, ('text')].split()
        words = [word for word in words if word[0] not in ('#', '@')]
        words = [word.replace('▪️', '') for word in words]        
                
        df.loc[i, ('text')] = ' '.join(words)
        # break
    # print(count)
    return df
    

Can't figure out why the function 
```
removing_hashtags_mentions(dataframe)
```
needs to be called multiple times instead of once to remove all the hashtags and mentions from the tweets

In [18]:

# print(tweets)
tweets_df = removing_hashtags_mentions(tweets_df)
tweets_df.head()
# print(tweets)

,user_id,tweet_id,username,location,following,followers,twt_created_at,total_tweets,retweet_count,text,hashtags,mentions
0,100673521,1585301310026711041,sai1951,NaN,396,81,2022-10-26 16:04:20+00:00,8226,0,Covid numbers in Karnataka: New cases in State...,"[{'text': 'Covid_19', 'indices': [392, 401]}, ...","[{'screen_name': 'DHFWKA', 'name': ""K'taka Hea..."
1,101430927,1585255112788017153,shivaramsingh,Cuttack,777,1242,2022-10-26 13:00:46+00:00,5578,0,Great! Well done Team PGI,[],"[{'screen_name': 'nipun29j', 'name': 'nipun ve..."
2,1550427109,1585252333013061634,rhltiwari711,"Lucknow, India",2573,520,2022-10-26 12:49:43+00:00,4641,0,Dear kindly provide to Prof Founder &amp; Head...,"[{'text': 'VC', 'indices': [74, 77]}, {'text':...","[{'screen_name': 'TwitterIndia', 'name': 'Twit..."
3,190904456,1585231415532490753,amishradp,"दिल्ली, भारत",51,9,2022-10-26 11:26:36+00:00,243,0,Listening these words from 20th october no one...,[],"[{'screen_name': 'care_mediassist', 'name': 'S..."
4,350945055,1585230709547237379,chouhanneeraj07,"Mumbai, India",879,488,2022-10-26 11:23:47+00:00,3450,0,"Totally wastage of time and efforts, most of t...",[],"[{'screen_name': 'drchetandeshmu1', 'name': 'd..."


In [19]:
def removing_punctuations(df):
    
    for i in range(len(df)):
            
        words = df.loc[i, ('text')]
        words =  words.translate(str.maketrans('', '', string.punctuation))
        df.loc[i, ('text')] = words
    
    return df


In [20]:
tweets_df = removing_punctuations(tweets_df)
tweets_df.head()

,user_id,tweet_id,username,location,following,followers,twt_created_at,total_tweets,retweet_count,text,hashtags,mentions
0,100673521,1585301310026711041,sai1951,NaN,396,81,2022-10-26 16:04:20+00:00,8226,0,Covid numbers in Karnataka New cases in State ...,"[{'text': 'Covid_19', 'indices': [392, 401]}, ...","[{'screen_name': 'DHFWKA', 'name': ""K'taka Hea..."
1,101430927,1585255112788017153,shivaramsingh,Cuttack,777,1242,2022-10-26 13:00:46+00:00,5578,0,Great Well done Team PGI,[],"[{'screen_name': 'nipun29j', 'name': 'nipun ve..."
2,1550427109,1585252333013061634,rhltiwari711,"Lucknow, India",2573,520,2022-10-26 12:49:43+00:00,4641,0,Dear kindly provide to Prof Founder amp Head H...,"[{'text': 'VC', 'indices': [74, 77]}, {'text':...","[{'screen_name': 'TwitterIndia', 'name': 'Twit..."
3,190904456,1585231415532490753,amishradp,"दिल्ली, भारत",51,9,2022-10-26 11:26:36+00:00,243,0,Listening these words from 20th october no one...,[],"[{'screen_name': 'care_mediassist', 'name': 'S..."
4,350945055,1585230709547237379,chouhanneeraj07,"Mumbai, India",879,488,2022-10-26 11:23:47+00:00,3450,0,Totally wastage of time and efforts most of th...,[],"[{'screen_name': 'drchetandeshmu1', 'name': 'd..."
